# 📊 Data Preparation

This notebook handles the full data preparation process:
- Reading service call data, events (holidays), and weather information
- Merging datasets by date
- Creating new featur


In [22]:
# Install required libraries if needed
# !pip install pandas matplotlib

import pandas as pd
import numpy as np


In [23]:
# Load Service Calls and Events
data = pd.read_csv('../data/data_filled.csv', low_memory=False)
events = pd.read_csv('../data/events_final.csv', low_memory=False)

print(f"Service Calls shape: {data.shape}")
print(f"Events shape: {events.shape}")

# Display first rows
data.head(), events.head()


Service Calls shape: (97159, 14)
Events shape: (821, 3)


(                אגף            מחלקה                    נושא  \
 0         אגף תפעול     מחלקת תברואה              אספקת פחים   
 1         אגף תפעול     מחלקת תברואה  דיווח על אי פינוי פחים   
 2         אגף תפעול       מדור קנסות      סוגיות חניה ודוחות   
 3  לשכת ראשת המועצה  לשכת ראשת מועצה       תלונות/בקשות כללי   
 4         אגף תפעול     מחלקת תברואה  דיווח על אי פינוי פחים   
 
                                            נושא משנה    סטטוס פנייה  \
 0                          תברואה - פח ירוק חדש/שבור  הטיפול הסתיים   
 1                          תברואה - אי פינוי פח ירוק  הטיפול הסתיים   
 2  חניה - בירורים על חוב / עיקול/דחיית ערעור/בירו...  הטיפול הסתיים   
 3                                       פניות תושבים  הטיפול הסתיים   
 4                          תברואה - אי פינוי פח כתום  הטיפול הסתיים   
 
   חריגה בשעות עבודה (פניות סגורות בלבד) אחוז חריגה (פניות סגורות בלבד)  \
 0                           55.32885683                    0.232215083   
 1                      

In [24]:
# Convert date columns to datetime
data['תאריך פתיחה'] = pd.to_datetime(data['תאריך פתיחה'], dayfirst=True, errors='coerce')
events['תאריך'] = pd.to_datetime(events['תאריך'], dayfirst=True, errors='coerce')


In [26]:
# Merge data with events
merged = pd.merge(
    data,
    events,
    left_on='תאריך פתיחה',
    right_on='תאריך',
    how='left'
)

# Drop duplicate 'תאריך' column
merged.drop(columns=['תאריך'], inplace=True)

# Save intermediate merged file
merged.to_csv('../data/data_merged.csv', index=False, encoding="utf-8-sig")

print("✅ Merged Service Calls + Events")


✅ Merged Service Calls + Events


In [29]:
# Merge merged service calls with weather
final = pd.merge(
    merged,
    weather,
    on='תאריך פתיחה',
    how='left'
)

# Save the final merged dataset
final.to_csv('../data/data_final_with_weather.csv', index=False, encoding="utf-8-sig")

print("✅ Merged with Weather Data")


✅ Merged with Weather Data


In [35]:
# Load the final merged data
data = pd.read_csv('../data/data_final_with_weather.csv', low_memory=False)

# Convert 'תאריך סגירה' to datetime
data['תאריך סגירה'] = pd.to_datetime(data['תאריך סגירה'], dayfirst=True, errors='coerce')

# Create separate date and time columns
data['שעת סגירה'] = data['תאריך סגירה'].dt.strftime('%H:%M')
data['תאריך סגירה'] = data['תאריך סגירה'].dt.strftime('%d/%m/%Y')

# Save after splitting
data.to_csv('../data/data_final_with_weather1.csv', index=False, encoding="utf-8-sig")

print("✅ Split 'תאריך סגירה' into 'תאריך סגירה' (date) and 'שעת סגירה' (time)")


✅ Split 'תאריך סגירה' into 'תאריך סגירה' (date) and 'שעת סגירה' (time)


In [36]:
# Load updated data
data = pd.read_csv('../data/data_final_with_weather1.csv', low_memory=False)

# Fix column names if needed
data.columns = data.columns.str.strip()

# Convert dates and times
data['תאריך פתיחה'] = pd.to_datetime(data['תאריך פתיחה'], dayfirst=True, errors='coerce')
data['תאריך סגירה'] = pd.to_datetime(data['תאריך סגירה'], dayfirst=True, errors='coerce')

data['שעת פתיחה'] = pd.to_datetime(data['שעת פתיחה'], format='%H:%M', errors='coerce').dt.time
data['שעת סגירה'] = pd.to_datetime(data['שעת סגירה'], format='%H:%M', errors='coerce').dt.time

# Function to safely combine date and time
def safe_combine(date, time):
    if pd.notna(date) and pd.notna(time):
        return pd.Timestamp.combine(date, time)
    else:
        return pd.NaT

# Apply combination
data['פתיחה - datetime'] = data.apply(lambda row: safe_combine(row['תאריך פתיחה'], row['שעת פתיחה']), axis=1)
data['סגירה - datetime'] = data.apply(lambda row: safe_combine(row['תאריך סגירה'], row['שעת סגירה']), axis=1)

# Calculate Treatment Duration
data['משך טיפול (בשעות)'] = (data['סגירה - datetime'] - data['פתיחה - datetime']).dt.total_seconds() / 3600

# Save
data.to_csv('../data/data_with_duration.csv', index=False, encoding="utf-8-sig")

print("✅ Calculated Treatment Duration")


✅ Calculated Treatment Duration


In [38]:
# Add weekday column (in Hebrew)
data['תאריך פתיחה'] = pd.to_datetime(data['תאריך פתיחה'], errors='coerce')

# Get weekday name
data['יום בשבוע'] = data['תאריך פתיחה'].dt.day_name()

# Translate to Hebrew
translation = {
    'Sunday': 'ראשון',
    'Monday': 'שני',
    'Tuesday': 'שלישי',
    'Wednesday': 'רביעי',
    'Thursday': 'חמישי',
    'Friday': 'שישי',
    'Saturday': 'שבת'
}
data['יום בשבוע'] = data['יום בשבוע'].map(translation)

# Save final file
data.to_csv('../data/file_with_weekday.csv', index=False, encoding="utf-8-sig")

print("✅ Added Hebrew Weekday Column - Final dataset ready!")


✅ Added Hebrew Weekday Column - Final dataset ready!


# Summary

The dataset has been successfully:
- Cleaned
- Enhanced with new features (day of the week, treatment duration)
- Scaled for model training

We also explored the distributions of key features, providing initial insights into the data behavior.
